In [15]:
def foo(a, b):
    if a == 0:
        #print("a = 0")
        return int(b)
    
    elif b == 0: 
        #print("b = 0")
        return int(a)
    
    else:
        #let (x,y) = (min(a,b), max(a,b))
        (x,y) = (min(a,b), max(a,b))

        #print(x,y)

        #recursive call, need to use return
        return foo(y, y % x)


returned_value = foo(1,2)

print(returned_value)

2


In [ ]:
#slow and wasteful but functional
def longest_run(mylist, key):
    counters = []
    for x in range(len(mylist)):

        if mylist[x] == key:
            counter += 1
            counters.append(counter)

        else:
            counter = 0

    return max(counters)

print(longest_run([2,12,12,8,12,12,12,0,12,1], 12))

longest_run([2,12,12,8,12,12,12,0,12,1], 12) == 3

3


True